In [1]:
print("Scenario A: Uneven weights with remainder distribution")

Scenario A: Uneven weights with remainder distribution


In [2]:
def format_as_table(split_result, currency=""):
    """
    Formats the split result as a neat text table.
    """

    # Determine column widths
    name_width = max(len(name) for name in split_result.keys())
    amount_width = max(len(f"{amount:.2f}") for amount in split_result.values())

    header = f"{'Name'.ljust(name_width)} | {'Amount'.rjust(amount_width)}"
    separator = "-" * len(header)

    lines = [header, separator]

    for name, amount in split_result.items():
        lines.append(
            f"{name.ljust(name_width)} | {currency}{amount:>{amount_width}.2f}"
        )

    return "\n".join(lines)

def split_expense(total, names, weights=None, round_to=2, distribute_remainder=True):
    """
    Splits a total expense among people with optional uneven weights and
    deterministic remainder distribution.

    Args:
        total (float): Total bill amount
        names (list): List of participant names
        weights (list or dict, optional): Weights per person
        round_to (int): Decimal places (default 2)
        distribute_remainder (bool): Distribute leftover cents if True

    Returns:
        dict: Mapping of name -> amount to pay
    """

    # ---------- Type checks ----------
    if not isinstance(total, (int, float)):
        raise TypeError("Total must be a number.")

    if not isinstance(names, list) or not names:
        raise ValueError("Names must be a non-empty list.")

    if not all(isinstance(name, str) for name in names):
        raise TypeError("Each name must be a string.")

    if not isinstance(round_to, int) or round_to < 0:
        raise ValueError("round_to must be a non-negative integer.")

    if weights is not None and not isinstance(weights, (list, dict)):
        raise TypeError("Weights must be a list or dict.")

    # ---------- Prepare weights ----------
    if weights is None:
        weights = {name: 1 for name in names}

    if isinstance(weights, list):
        if len(weights) != len(names):
            raise ValueError("Weights list length must match names.")
        if not all(isinstance(w, (int, float)) and w > 0 for w in weights):
            raise ValueError("All weights must be positive numbers.")
        weights = dict(zip(names, weights))

    for name in names:
        if name not in weights or weights[name] <= 0:
            raise ValueError(f"Invalid weight for {name}.")

    # ---------- Core calculation ----------
    scale = 10 ** round_to
    total_scaled = int(round(total * scale))
    total_weight = sum(weights.values())

    # Step 1: calculate unrounded shares
    raw_shares = {
        name: (weights[name] / total_weight) * total_scaled
        for name in names
    }

    # Step 2: floor all shares
    split_scaled = {name: int(raw_shares[name]) for name in names}

    # Step 3: compute remainder
    remainder = total_scaled - sum(split_scaled.values())

    # Step 4: distribute remainder deterministically
    if distribute_remainder:
        for name in names:
            if remainder <= 0:
                break
            split_scaled[name] += 1
            remainder -= 1

    # Step 5: convert back to float
    return {
        name: split_scaled[name] / scale
        for name in names
    }

total = 500
names = ["Alice", "Bob", "Charlie"]
weights = {"Alice": 1, "Bob": 2, "Charlie": 1}

result = split_expense(total, names, weights=weights, round_to=2)
result = format_as_table(result, currency="₹")
print(result)

########################
total = 1000
names = ["Rahul", "Anita", "Vikram"]
weights = {"Rahul": 2, "Anita": 2, "Vikram": 2}
result = split_expense(total, names, weights=weights, round_to=2)
result = format_as_table(result, currency="₹")
print("• Add option to handle remainders (distribute extra cents to first N people deterministically)")
print(result)
########################
total = 750
names = ["Solo"]
result = split_expense(total, names)
result = format_as_table(result, currency="₹")
print(result)


#########################
# ✅ Error Handling Demo - Invalid Types
print("\n--- Error Handling Examples ---\n")

# Test Case 1: Invalid input types (string instead of number and list)
try:
    print("Test 1: Invalid types (total='rahul', names='Anita')")
    total = "rahul" 
    names = "Anita"
    result = split_expense(total, names)
    result = format_as_table(result, currency="₹")
    print(result)
except TypeError as e:
    print(f"✅ TypeError caught: {e}\n")
except ValueError as e:
    print(f"✅ ValueError caught: {e}\n")

# Test Case 2: Negative total
try:
    print("Test 2: Negative total amount")
    total = -500
    names = ["Alice", "Bob"]
    result = split_expense(total, names)
    result = format_as_table(result, currency="₹")
    print(result)
except ValueError as e:
    print(f"✅ ValueError caught: {e}\n")

# Test Case 3: Empty names list
try:
    print("Test 3: Empty names list")
    total = 1000
    names = []
    result = split_expense(total, names)
    result = format_as_table(result, currency="₹")
    print(result)
except ValueError as e:
    print(f"✅ ValueError caught: {e}\n")

# Test Case 4: Invalid weights (negative value)
try:
    print("Test 4: Negative weight values")
    total = 10000
    names = ["Alice", "Bob"]
    weights = {"Alice": -5, "Bob": 2}
    result = split_expense(total, names, weights=weights)
    result = format_as_table(result, currency="₹")
    print(result)
except ValueError as e:
    print(f"✅ ValueError caught: {e}\n")

# Test Case 5: Mixed valid and invalid scenario
try:
    print("Test 5: Type mismatch in names list")
    total = 500
    names = ["Alice", 123, "Charlie"]  # 123 is not a string
    result = split_expense(total, names)
    result = format_as_table(result, currency="₹")
    print(result)
except TypeError as e:
    print(f"✅ TypeError caught: {e}\n")




Name    | Amount
----------------
Alice   | ₹125.00
Bob     | ₹250.00
Charlie | ₹125.00
• Add option to handle remainders (distribute extra cents to first N people deterministically)
Name   | Amount
---------------
Rahul  | ₹333.34
Anita  | ₹333.33
Vikram | ₹333.33
Name | Amount
-------------
Solo | ₹750.00

--- Error Handling Examples ---

Test 1: Invalid types (total='rahul', names='Anita')
✅ TypeError caught: Total must be a number.

Test 2: Negative total amount
Name  |  Amount
---------------
Alice | ₹-250.00
Bob   | ₹-250.00
Test 3: Empty names list
✅ ValueError caught: Names must be a non-empty list.

Test 4: Negative weight values
✅ ValueError caught: Invalid weight for Alice.

Test 5: Type mismatch in names list
✅ TypeError caught: Each name must be a string.

